<a href="https://colab.research.google.com/github/SilahicAmil/NLP-NLTK/blob/main/Tweet_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Shakespeare Hamlet Text

In [74]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Sklearn
from sklearn.model_selection import train_test_split

# Tesnorflow
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#Keras
import keras

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
train_df = pd.read_csv("/content/drive/MyDrive/Tweet_Sentiment/tweet-sentiment-extraction/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Tweet_Sentiment/tweet-sentiment-extraction/test.csv")

In [41]:
train_df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


## Preprocessing

In [42]:
train_df = train_df.drop("selected_text", 1)

In [43]:
train_df.head()

,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
2,088c60f138,my boss is bullying me...,negative
3,9642c003ef,what interview! leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative


In [56]:
# Dropping all neutral values
train_df = train_df[train_df.sentiment != "neutral"]
# Replacing pos and neg with numerical values
train_df_copy = train_df.copy()
mapping = {'positive': 1, 'negative': 0}

train_df_copy = train_df_copy.replace({'sentiment': mapping})

In [57]:
train_df_copy.head()

,textID,text,sentiment
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0
2,088c60f138,my boss is bullying me...,0
3,9642c003ef,what interview! leave me alone,0
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,1


Postivie =1

Negative = 0

## Shuffle Data

In [59]:
# Shuffle df
shuffled_train_df = train_df_copy.sample(frac=1, random_state=42)
shuffled_train_df.head()

,textID,text,sentiment
14813,508435b32d,thanks will try to behave,1
18134,de3833a81d,Today is lame because I am not in Orlando I a...,0
9074,85a65f0656,hahahaha! i laughed my **** off just now. thanks,1
23126,d70ed4038d,Gahh ! This weather sucksss !,0
16935,1b4d9119e9,Fallen in love with enter shikari again. Might...,1


In [60]:
# Test data
test_df.head()
test_df = test_df.drop("sentiment", 1)


In [61]:
test_df.head()

,textID,text
0,f87dea47db,Last session of the day http://twitpic.com/67ezh
1,96d74cb729,Shanghai is also really exciting (precisely -...
2,eee518ae67,"Recession hit Veronique Branquinho, she has to..."
3,01082688c6,happy bday!
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!


In [63]:
train_df_copy.sentiment.value_counts()

1    8582
0    7781
Name: sentiment, dtype: int64

In [64]:
len(train_df_copy), len(test_df)

(16363, 3534)

## Validation Set Creation

In [65]:
# Creating val set

train_sentences, val_sentences, train_labels, val_labels = train_test_split(shuffled_train_df["text"].to_numpy(),
                                                                            shuffled_train_df["sentiment"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [66]:
len(train_sentences), len(train_labels)

(14726, 14726)

In [67]:
len(val_sentences), len(val_labels)

(1637, 1637)

In [68]:
train_sentences[:10], train_labels[:10]

(array(['Great find   http://amanita-design.net/samorost-1/',
        ' everyone loves u sarah not just the tweeters! the today show couldnt stop raving about your beautifulness...',
        'Hello dark hair! Well, my plans for today just got cancelled  make some better ones?!',
        'just watched devil wears prada, and i want to live in new york city more than ever. why must i be so poor',
        'Is feeling really bad about goofin` on  not knowing she really wasn`t at the meeting! Dang I`m sorry! Me & my big `ol mouth',
        'im blonding ... slowly but surely im blonding ... i wana scratch my hair but im not allowed',
        ' man i need 2 find a siitter Val still aint **** wit me  LOL',
        ' its my birthday too  but its the 10th of may right now for me.. and its probably the 9th for you.. happy birthday anyway',
        'Good morning all! Its a fabulous morning here!', '  nice one'],
       dtype=object), array([0, 1, 0, 0, 0, 0, 0, 1, 1, 1]))

## Converting Text to Numbers

Standarize each sample (lowercase + punctucation stripping)

Split each sample into substrings (usually words)

Recombine substrings to tokens (usually ngrams)

Index tokens (unique int value > Token)

Transform each sample with index into vector or dense float vector



In [81]:
# Find average number of tokens 
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

13.318823848974603

In [83]:
# Setting text vectorization variable
MAX_VOCAB_LEN = 10000 # Max words in vocab
MAX_LEN = 13 # MAX LEN OF SEQUENCE

txt_vect = TextVectorization(max_tokens=MAX_VOCAB_LEN,
                              output_mode="int",
                              output_sequence_length=MAX_LEN)